 This notebook was created by Florian Keibel to teach himself about the world of data science.

# Using Data to find a fitting spot for a new Subway restaurant in Lower Manhattan

## 1 Introduction / Business problem

"Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem."

A business entrepreneur wants to open a new Subway in NYC. He is competing in a high competitive environment and is currently interested in five different locations, where he could buy property to open a new Subway. The entrepreneur now comes to us to find the ideal location of his five choices. He asks us, which location is best considering:
- distance to competing Subways (he is confident of his business model, but a little less competition never hurts)
- distance to public transport (so more customers can enjoy the Subway or are passing by) 
- and distance to others restaurants.



## 2. Data

### 2.1 What dala will we need?
We will use data of the city of New York for Manhattan retailers to get an overview about existing subways in New York.
You can view the data here:

https://data.cityofnewyork.us/resource/uyz2-yxi9.csv

We will also use the foursquare api to gain knowledge of venues near the locations.

### 2.2 Data Aquisition

#### 2.2.1 Subway Data in Lower Manhattan

As we already mentioned, we will use an open data set from the city of New York. The data set contains information about existing retailers in Lower Manhattan with information regarding their adress, geospatial data, name, type of retailer etc.

In [1]:
# Import necessary library
import json
import pandas as pd

In [2]:
!wget -O retail.csv https://data.cityofnewyork.us/resource/uyz2-yxi9.csv

--2019-01-25 23:52:59--  https://data.cityofnewyork.us/resource/uyz2-yxi9.csv
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘retail.csv’

    [ <=>                                   ] 194,481     --.-K/s   in 0.07s   

2019-01-25 23:53:00 (2.82 MB/s) - ‘retail.csv’ saved [194481]



In [3]:
# We will now read the csv file into a dataframe
df = pd.read_csv("retail.csv")
df.head()

,bbl,bin,block_lot,borough,census_tract,cnadrprf_addrline1,cnadrprf_addrline2,cnadrprf_city,cnadrprf_state,cnadrprf_zip,cnbio_org_name,community_board,council_district,latitude,longitude,nta,primary,secondary
0,NaN,NaN,Aug-32,NaN,NaN,3 New York Plaza,NaN,New York,NY,10004.0,Starbucks Coffee,NaN,NaN,NaN,NaN,NaN,Casual Eating & Takeout,F-Coffeehouse
1,1.000088e+09,1087700.0,Aug-32,MANHATTAN,9.0,39 Whitehall Street,NaN,New York,NY,10004.0,New York Health & Racquet Club,1.0,1.0,40.703037,-74.012969,Battery Park City-Lower Manhattan ...,Personal and Professional Services,P-Athletic Clubs/Fitness
2,1.000100e+09,1000028.0,Oct-32,MANHATTAN,9.0,6 Stone Street,NaN,New York,NY,10004.0,A.J. Kelly's,1.0,1.0,40.704028,-74.012692,Battery Park City-Lower Manhattan ...,Full Service Dining,F-Irish Pub
3,1.000070e+09,1078965.0,Jul-35,MANHATTAN,9.0,54 Pearl Street,NaN,New York,NY,10004.0,Fraunces Tavern® Restaurant,1.0,1.0,40.703496,-74.011375,Battery Park City-Lower Manhattan ...,Full Service Dining,F-American
4,1.000070e+09,1000014.0,Jul-37,MANHATTAN,9.0,60 Pearl Street,NaN,New York,NY,10004.0,Bombay's,1.0,1.0,40.703553,-74.011152,Battery Park City-Lower Manhattan ...,Casual Eating & Takeout,F-Indian


We are only interested in the Subway restaurants in this dataset. So we filter for them

In [4]:
subways = df.loc[df['cnbio_org_name'] == "Subway"]
subways.head()

,bbl,bin,block_lot,borough,census_tract,cnadrprf_addrline1,cnadrprf_addrline2,cnadrprf_city,cnadrprf_state,cnadrprf_zip,cnbio_org_name,community_board,council_district,latitude,longitude,nta,primary,secondary
16,NaN,NaN,1-Apr,NaN,NaN,1 New York Plaza,Concourse,New York,NY,10004.0,Subway,NaN,NaN,NaN,NaN,NaN,Casual Eating & Takeout,F-Fast Food
21,1.000070e+09,1000009.0,28-Jul,MANHATTAN,9.0,28 Water Street,NaN,New York,NY,10004.0,Subway,1.0,1.0,40.703095,-74.011029,Battery Park City-Lower Manhattan ...,Casual Eating & Takeout,F-Fast Food
118,1.000130e+09,1000044.0,13-5,MANHATTAN,13.0,11 Broadway,NaN,New York,NY,10004.0,Subway,1.0,1.0,40.705354,-74.013731,Battery Park City-Lower Manhattan ...,Casual Eating & Takeout,F-Fast Food
184,1.000240e+09,1000819.0,24-29,MANHATTAN,9.0,30 New Street,NaN,New York,NY,10004.0,Subway,1.0,1.0,40.706314,-74.011978,Battery Park City-Lower Manhattan ...,Casual Eating & Takeout,F-Fast Food
317,1.000390e+09,1000882.0,39-40,MANHATTAN,7.0,122 Water Street,NaN,New York,NY,10005.0,Subway,1.0,1.0,40.705343,-74.007181,Battery Park City-Lower Manhattan ...,Casual Eating & Takeout,F-Fast Food


We drop some columns we will not need for our analysis

In [5]:
# We could have also just splice the dataframe but i am still unfamiliar with the drop function, so i applied it here
sub_drop = subways.drop(['block_lot','bbl','bin','census_tract','cnadrprf_city','cnadrprf_zip',
              'community_board', 'cnadrprf_addrline2','council_district', 'primary', 'secondary', 'cnadrprf_state', 'nta'], axis = 1)
sub_drop.reset_index(inplace=True, drop=True)
sub_drop

,borough,cnadrprf_addrline1,cnbio_org_name,latitude,longitude
0,NaN,1 New York Plaza,Subway,NaN,NaN
1,MANHATTAN,28 Water Street,Subway,40.703095,-74.011029
2,MANHATTAN,11 Broadway,Subway,40.705354,-74.013731
3,MANHATTAN,30 New Street,Subway,40.706314,-74.011978
4,MANHATTAN,122 Water Street,Subway,40.705343,-74.007181
5,MANHATTAN,100 Maiden Lane,Subway,40.706867,-74.006990
6,MANHATTAN,111 Broadway,Subway,40.708486,-74.011113
7,MANHATTAN,106 Greenwich Street,Subway,40.708442,-74.013400
8,MANHATTAN,21 Maiden Lane,Subway,40.709260,-74.009082
9,MANHATTAN,112 John Street,Subway,40.707676,-74.005796


Now we See we have one line where we have NaN values. Since only one value is missing we can try to add the values manually. 
With the help of latlong.net, we see that the values are 

40.702194,-74.012046

We also add the missing borough to our data.

In [6]:
values = {'borough': 'MANHATTAN','latitude': 40.702194, 'longitude': -74.012046}
sub_clean = sub_drop.fillna(value=values)
sub_cleaned= sub_clean.rename(index=str, columns={"cnadrprf_addrline1": "adress", "cnbio_org_name": "name"})
sub_cleaned.head()

,borough,adress,name,latitude,longitude
0,MANHATTAN,1 New York Plaza,Subway,40.702194,-74.012046
1,MANHATTAN,28 Water Street,Subway,40.703095,-74.011029
2,MANHATTAN,11 Broadway,Subway,40.705354,-74.013731
3,MANHATTAN,30 New Street,Subway,40.706314,-74.011978
4,MANHATTAN,122 Water Street,Subway,40.705343,-74.007181


In [7]:
sub_cleaned['exists'] = '1'

sub_cleaned = sub_cleaned.append({'borough': "MANHATTAN", 'adress': "55 Wall Street", 'name' : "Subway", 'latitude': 40.706019, "longitude": -74.008831 , 'exists': 0}, ignore_index=True)
sub_cleaned = sub_cleaned.append({'borough': "MANHATTAN", 'adress': "Battery Place Market" , 'name' : "Subway", 'latitude': 40.707224, "longitude":-74.017063 , 'exists': 0}, ignore_index=True)
sub_cleaned = sub_cleaned.append({'borough': "MANHATTAN", 'adress': "World Trace Center", 'name' : "Subway", 'latitude':40.711425 , "longitude": -74.012413, 'exists': 0}, ignore_index=True)


In [8]:
import folium

# We need lat and long matching Lower Manhattan, since we will need a zoomed-in view of our map
latitude=40.705
longitude=-74.011

map_nyc = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, long, name, borough, address, exists in zip(sub_cleaned['latitude'], sub_cleaned['longitude'], sub_cleaned['name'],
                                           sub_cleaned['borough'], sub_cleaned['adress'], sub_cleaned['exists']):
    label = "{} ({}): {} , Exists:{}".format(borough, name, address, exists)
    popup = folium.Popup(label, parse_html=True)
    if exists == 0:
        colorCircle = "blue"
        filled_color = "#4442cc"
    else:
        colorCircle = "red"
        filled_color = "#cc4f41"
    folium.CircleMarker(
        [lat, long],
        radius=7,
        popup=popup,
        color=colorCircle,
        fill=True,
        fill_color=filled_color,
        fill_opacity=0.5,
        parse_html=False).add_to(map_nyc)
    
map_nyc

### 2.2.2 Using foursquare API to receive venue data

In [10]:
sub_potentials = sub_cleaned.loc[sub_cleaned['exists'] == 0]
sub_potentials

,borough,adress,name,latitude,longitude,exists
11,MANHATTAN,55 Wall Street,Subway,40.706019,-74.008831,0
12,MANHATTAN,Battery Place Market,Subway,40.707224,-74.017063,0
13,MANHATTAN,World Trace Center,Subway,40.711425,-74.012413,0


In [11]:
# The code was removed by Watson Studio for sharing.

In [24]:
def getNearbyVenues(latitudes, longitudes,adresses, radius=500, LIMIT=100):
    
    venues_list=[]
    for  lat, lng,adress in zip(latitudes, longitudes,adresses):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            adress, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Adress', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
import requests
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(latitudes=sub_potentials['latitude'],
                                   longitudes=sub_potentials['longitude'],
                                    adresses=sub_potentials["adress"]
                                  )

In [26]:
location_venues.head()

,Adress,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,55 Wall Street,40.706019,-74.008831,sweetgreen,40.705586,-74.008382,Salad Place
1,55 Wall Street,40.706019,-74.008831,Black Fox Coffee Co.,40.706573,-74.008155,Coffee Shop
2,55 Wall Street,40.706019,-74.008831,Manhatta,40.707654,-74.009138,New American Restaurant
3,55 Wall Street,40.706019,-74.008831,Cipriani Wall Street,40.706181,-74.009284,Event Space
4,55 Wall Street,40.706019,-74.008831,Fearless Girl,40.706826,-74.010783,Monument / Landmark


In [27]:
location_venues.groupby("Adress").count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Adress,,,,,,
55 Wall Street,100,100,100,100,100,100
Battery Place Market,76,76,76,76,76,76
World Trace Center,100,100,100,100,100,100


We can see we have reached our max. Venue count for the potential Wall Street and World Trade Center location. In reality there are more, but we want to limit our calls to the 
foursquare api and just will assume here that these 100 venues are all venues in the proximity of the potential Subways.

In [29]:
# hot encoding
venues_hot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_hot['Adress'] = location_venues['Adress'] 

# move street column to the first column
fixed_columns = [venues_hot.columns[-1]] + list(venues_hot.columns[:-1])

#fixed_columns
venues_hot = venues_hot[fixed_columns]

venues_hot.head()

,Adress,Accessories Store,American Restaurant,Asian Restaurant,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Garden,...,Spa,Sporting Goods Shop,Steakhouse,Strip Club,Supplement Shop,Tree,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,55 Wall Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,55 Wall Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,55 Wall Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,55 Wall Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,55 Wall Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
potentials_grouped = venues_hot.groupby('Adress').mean().reset_index()
potentials_grouped

,Adress,Accessories Store,American Restaurant,Asian Restaurant,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Garden,...,Spa,Sporting Goods Shop,Steakhouse,Strip Club,Supplement Shop,Tree,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,55 Wall Street,0.01,0.05,0.01,0.00,0.00,0.01,0.010000,0.020000,0.000000,...,0.020000,0.00,0.040000,0.00,0.000000,0.00,0.04,0.01,0.020000,0.000000
1,Battery Place Market,0.00,0.00,0.00,0.00,0.00,0.00,0.013158,0.013158,0.013158,...,0.013158,0.00,0.026316,0.00,0.013158,0.00,0.00,0.00,0.013158,0.013158
2,World Trace Center,0.00,0.01,0.01,0.01,0.01,0.00,0.020000,0.020000,0.000000,...,0.000000,0.01,0.020000,0.01,0.000000,0.01,0.01,0.01,0.020000,0.020000


In [39]:
num_top_venues = 10

for potential in potentials_grouped["Adress"]:
    print("---- Top Venues surrounding potential Subway :"+potential+"----")
    temp = potentials_grouped[potentials_grouped['Adress'] == potential].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Top Venues surrounding potential Subway :55 Wall Street----
                           venue  freq
0                    Coffee Shop  0.10
1            American Restaurant  0.05
2                     Steakhouse  0.04
3                            Gym  0.04
4           Gym / Fitness Center  0.04
5  Vegetarian / Vegan Restaurant  0.04
6                      Juice Bar  0.04
7             Falafel Restaurant  0.03
8                          Plaza  0.03
9                          Hotel  0.03


---- Top Venues surrounding potential Subway :Battery Place Market----
                venue  freq
0                Park  0.12
1               Hotel  0.07
2         Coffee Shop  0.05
3           Hotel Bar  0.03
4         Pizza Place  0.03
5        Gourmet Shop  0.03
6                 Gym  0.03
7       Historic Site  0.03
8  Italian Restaurant  0.03
9  Mexican Restaurant  0.03


---- Top Venues surrounding potential Subway :World Trace Center----
                  venue  freq
0           Coffee Shop 

In [40]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Adress']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [44]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Adress'] = potentials_grouped['Adress']

for ind in np.arange(potentials_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(potentials_grouped.iloc[ind, :], num_top_venues)

In [45]:
venues_sorted.head()

,Adress,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,55 Wall Street,Coffee Shop,American Restaurant,Juice Bar,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Steakhouse,Cocktail Bar,Italian Restaurant,Plaza
1,Battery Place Market,Park,Hotel,Coffee Shop,Museum,Italian Restaurant,Hotel Bar,Historic Site,Pizza Place,Gym,Gourmet Shop
2,World Trace Center,Coffee Shop,Hotel,Sandwich Place,Park,Clothing Store,Ice Cream Shop,Memorial Site,Market,Italian Restaurant,Indian Restaurant


Now for our potential spot in Wall Street we have 7 food venues in the top 10

For the Battery Place Market we have 3 out of 10

And for World Trace Center we have 5 out of 10, and one of them is already a Sandwich Place !!

Now all of this would speak for Battery Place Market. We can show our findings to the business entrepreneur. Thanks for reading!

